# 1. Introduction

The purpose of this section is to develop the necessary data processing steps to modify the dataset for subsequent data analysis. 

The data processing steps are developed with the following goals:
- The columns are in the correct data type
- The dataset does not contain outliers
- The dataset should contain newly created features

It is important to note that the focus of this section is to create a dataset for the next step which is data visualization.  

As more insight is obtained, the dataset will be further modified.

# 2. Data Collection

The data is collected from insideairbnb.com. It is an activist webiste that periodically scrap airbnb's database for listing information.

The following file is downloaded and unzipped into the file 'listings.csv' for analysis:  
http://data.insideairbnb.com/japan/kant%C5%8D/tokyo/2019-06-27/data/listings.csv.gz

According to the information provided, the data is scraped on 2019/06/27.

The file *listings.csv* can be downloaded from: https://drive.google.com/open?id=17A535-0gvKU_Q2Fyw7H3T3UFiXbeKWAj

# 3. Initial Dataset Exploration

In [1]:
# import required packages
import numpy as np
import pandas as pd
import re
import datetime
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('data/listings.csv')

C:\Users\georg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (61,62,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The warning message indicates two columns (61,62) have mixed data types.

In [2]:
df.neighbourhood.value_counts()

NameError: name 'df' is not defined

In [5]:
df.neighbourhood_cleansed.nunique()

140

In [12]:
df.neighbourhood_cleansed.isnull().sum()

0

In [3]:
#Inspect those columns
df.iloc[:,[61,62,95]].head()

,weekly_price,monthly_price,jurisdiction_names
0,"$2,000.00",NaN,NaN
1,NaN,"$2,679.00",NaN
2,$397.00,$973.00,NaN
3,NaN,NaN,NaN
4,NaN,"$4,766.00",NaN


It seems the mixed type is due to the dollar sign and NaN. This issue will be corrected in the data cleaning step.

First, explore the data set:

In [4]:
#Inspect number of rows and features
df.shape

(20769, 106)

There are 20,769 rows and and 106 features. Each row consists of a listing on airbnb in the Greater Toronto Area. 

The features can be divided into the following 6 subcategories:

1. **Host information**: interaction, host_id, host_url, host_name, host_since, host_location, host_about, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost, host_thumbnail_url, host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications, host_has_profile_pic, host_identity_verified
2. **Property information**: name, summary, space, description, neighborhood_overview, notes, transit, access, street, neighbourhood, neighbourhood_cleansed, neighbourhood_group_cleansed, city, state, zipcode, market, smart_location, country_code, country, latitude, longitude, is_location_exact, property_type, room_type, accommodates, bathrooms, bedrooms, beds, bed_type, amenities,square_feet,jurisdiction_names, requires_license, license
3. **Booking information and policy**: experiences_offered, house_rules, price, weekly_price, monthly_price,security_deposit, cleaning_fee, guests_included, extra_people, minimum_nights, maximum_nights, minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, calendar_updated,  has_availability, availability_30, availability_60, availability_90, availability_365, calendar_last_scraped, is_business_travel_ready, cancellation_policy, require_guest_profile_picture, require_guest_phone_verification, calculated_host_listings_count, calculated_host_listings_count_entire_homes, calculated_host_listings_count_private_rooms, calculated_host_listings_count_shared_rooms, instant_bookable
4. **Airbnb listing information**: id, listing_url, thumbnail_url, medium_url, picture_url, xl_picture_url 
5. **Reviews**: number_of_reviews, number_of_reviews_ltm, first_review, last_review, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, reviews_per_month
6. **Web scraping information**: scrape_id, last_scraped

    

It is likely that not all features will be used for data analysis. Nonetheless, all features are kept for now. Given the moderate size (14066 rows x 106 features) of this dataset, keeping all the columns should not significantly impact memory and computational speed.  

Irrelevant features will be removed at the last step of the data cleaning process. 
Note that the dataset may need further processing for statistical analysis and machine learning. 

# 4. Data Cleaning

## 4.1 Features with Excessive Missing Values Removal 

First, create a list of missing values for each row in the dataset.

Features with excessive number of missing values are considered unuseful for data analysis, and is therefore removed.  
For this analysis, a threshold of 50% is set; i.e., features with 50% or greater of the values missing will be removed.   

In [5]:
threshold = 0.5
remove_columns_NA = df.columns[df.isnull().sum()/df.shape[0] > threshold]
print(remove_columns_NA)

Index(['notes', 'thumbnail_url', 'medium_url', 'xl_picture_url',
       'host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet',
       'weekly_price', 'monthly_price', 'license', 'jurisdiction_names'],
      dtype='object')


In [6]:
df.drop(remove_columns_NA,axis=1,inplace=True)
df.shape

(20769, 95)

In [7]:
#Keep record of removed columns
removed_columns_record = []
removed_columns_record.append(remove_columns_NA)

After this step, 95 features remain.

## 4.2 Data Type Correction

The purpose of this section is to ensure the data types of each columns are correct. The focus will be on numeric and categorical data types.  

A few columns are boolean, but the values are stored as string type, with values of 't' or 'f'. For this part they are considered as categorical because that way they will be easier to manipuate in data visualization.

### 4.2.1 Change column data type to boolean 

In [8]:
# col_to_boolean = ['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability',
#                   'requires_license','instant_bookable','is_business_travel_ready','require_guest_profile_picture',
#                   'require_guest_phone_verification']

In [9]:
# # Change data type to boolean and add new column for NA values
# d = {'f':False, 't':True}
# for col in col_to_boolean:
#     # add new column if needed
#     if df[col].isnull().sum() > 0:
#         df[col+'_NA'] = np.where(df[col].isnull(),True,False)
#         m = df[col].mode().iloc[0]
#         df[col] = df[col].fillna(m).map(d).astype('bool')
#     else:
#         df[col]=df[col].map(d).astype('bool')

In [10]:
df.shape

(20769, 95)

### 4.2.2 Change column data type to numeric

A few numeric columns are in string format. It is necessary to convert them to numeric for data analysis.

In [11]:
col_to_numeric = ['price','security_deposit','cleaning_fee','extra_people']

In [12]:
for col in col_to_numeric:
    df[col] = df[col].str.replace('$','').str.replace(',','').astype(float)

In [13]:
df.host_response_rate = df.host_response_rate.str.replace('%','').astype(float)/100

### 4.2.3 Change column data type to datetime

A few columns consist of datatime information, but are stored as string. It is necessary to convert them to datetime data type.

In [14]:
col_to_datetime = ['last_scraped','host_since','calendar_last_scraped','first_review','last_review']

In [15]:
for col in col_to_datetime:
    df[col] = pd.to_datetime(df[col])

### 4.2.4 Change column data type to category

Some columns are categorical but are stored as string. It is necessary to convert them to categorial data type.

In [16]:
col_to_cat = ['host_response_time','property_type','room_type','bed_type','cancellation_policy']

#boolean data types; convert to category
col_to_cat2 = ['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability',
                  'requires_license','instant_bookable','is_business_travel_ready','require_guest_profile_picture',
                  'require_guest_phone_verification']

In [17]:
for col in col_to_cat:
    df[col] = df[col].astype('category')
for col in col_to_cat2:
    df[col]=df[col].astype("category")

## 4.3 Extreme Values Identification

It is important to ensure the data is accurate, and the presence of outliers is an indicator of inaccuarcy. In this section, outliers are identified and investigated. 

In [18]:
numeric_features = [col for col in df.columns if df[col].dtype in ['int64','float64']]

In [19]:
print(numeric_features)

['id', 'scrape_id', 'host_id', 'host_response_rate', 'host_listings_count', 'host_total_listings_count', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month']


The statistics of the data are inspected with the _describe_ method. Only data likely to be relevant to data analysis will be reviewed.

### 4.3.1 Nightly prices and fees

In [20]:
df.loc[:,['price', 'security_deposit', 'cleaning_fee', 'extra_people']].describe()

,price,security_deposit,cleaning_fee,extra_people
count,20769.000000,15867.000000,17385.000000,20769.000000
mean,143.288074,327.501859,60.276445,14.162694
std,234.157135,452.885745,49.732881,23.543072
min,0.000000,0.000000,0.000000,0.000000
25%,64.000000,130.000000,25.000000,0.000000
50%,101.000000,250.000000,50.000000,8.000000
75%,160.000000,500.000000,80.000000,20.000000
max,13422.000000,6711.000000,962.000000,403.000000


It can be seen that the maximum nightly price is \$13,422 which is very high. To ensure accuracy, listings with price over \$5,000 is further investigated.

In [21]:
df.loc[df.price > 5000.00,['id','listing_url','name','price']]

,id,listing_url,name,price
6263,16039481,https://www.airbnb.com/rooms/16039481,Art Collector's Penthouse,13422.0
6854,17330866,https://www.airbnb.com/rooms/17330866,Toronto's Bloor West Village - High Park - B,5900.0
8974,20653172,https://www.airbnb.com/rooms/20653172,Modern Upscale Condo in downtown Toronto,13000.0
19221,33983450,https://www.airbnb.com/rooms/33983450,Executive family home near Casa Loma and ravines.,7500.0
19783,34470470,https://www.airbnb.com/rooms/34470470,NEW! Executive 1 BR+Den Condo in Heart of Down...,9999.0
19787,34471108,https://www.airbnb.com/rooms/34471108,1BR + 3 Queen BED. Sleeps 6 Condo in MapleLeaf...,9999.0


The URLs of the six listings are investigated, with the prices recorded below.

In [22]:
actual_price = [13078,78,13000,7500,174,224]
df_high_price = df.loc[df.price > 5000.00,['id','listing_url','name','price']]
df_high_price['actual_price'] = actual_price
df_high_price

,id,listing_url,name,price,actual_price
6263,16039481,https://www.airbnb.com/rooms/16039481,Art Collector's Penthouse,13422.0,13078
6854,17330866,https://www.airbnb.com/rooms/17330866,Toronto's Bloor West Village - High Park - B,5900.0,78
8974,20653172,https://www.airbnb.com/rooms/20653172,Modern Upscale Condo in downtown Toronto,13000.0,13000
19221,33983450,https://www.airbnb.com/rooms/33983450,Executive family home near Casa Loma and ravines.,7500.0,7500
19783,34470470,https://www.airbnb.com/rooms/34470470,NEW! Executive 1 BR+Den Condo in Heart of Down...,9999.0,174
19787,34471108,https://www.airbnb.com/rooms/34471108,1BR + 3 Queen BED. Sleeps 6 Condo in MapleLeaf...,9999.0,224


For listing id 17330866, 34470470, and 34471108, there is a huge discrepancy between the listed price and the actual price found on the web. As such the three rows are removed.

In [23]:
# drop by index

### Don't drop in this section

# for idx in (6854,19783,19787):
#     df.drop(idx,inplace=True)

In [24]:
# Keep record of dropped columns

### Don't drop in this section

# removed_indexes_record = []
# removed_indexes_record.append([6854,19783,19787])

The security deposit shows a wide range, from \$0 up to \$6,711. It is an option available for the hosts to protect their properties, with the amount chosen by the host. Note that it is not the amount the customers need to pay upfront, but rather the max amount the host can claim in case of a dispute.

The cleaning fee shows a wide range, from \$0 up to \$403. It is a one-time charge for every booking.  

The range of charge for additional guest (extra_people) is between \$0 and \$403. It is the charge applied for extra guests.  

Since those values are decided by the host, they are not outliers.

### 4.3.2 Host Information

In [25]:
df.loc[:,['host_listings_count', 'host_total_listings_count','host_response_rate']].describe()

,host_listings_count,host_total_listings_count,host_response_rate
count,20764.000000,20764.000000,15785.000000
mean,5.613225,5.613225,0.948670
std,12.654742,12.654742,0.152756
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000
50%,2.000000,2.000000,1.000000
75%,4.000000,4.000000,1.000000
max,179.000000,179.000000,1.000000


The minimum host listings count is 0, which is counter-intuitive because the minimum count should be 1 for having a listing. The definition of this feature need to be explored.  

Also, it is suspected the two columns 'host_listings_count' and 'host_total_listings_count' are identical. If identical, the latter column will be dropped.

In [26]:
### Don't drop here


# if df.host_listings_count.equals(df.host_total_listings_count) == True:
#     print('The 2nd column is dropped because they are identical.')
#     df=df.drop('host_total_listings_count',axis=1)

In [27]:
#keep record of removed columns
removed_columns_record.append(['host_total_listings_count'])

### 4.3.3 Geographical locations

In [28]:
df.loc[:,['latitude', 'longitude']].describe()

,latitude,longitude
count,20769.000000,20769.000000
mean,43.679969,-79.397500
std,0.048124,0.061759
min,43.586710,-79.623460
25%,43.644970,-79.424420
50%,43.662320,-79.397420
75%,43.696420,-79.377970
max,43.831710,-79.123400


The latitude and longitude values seem legit.

### 4.3.4 Property information

In [29]:
df.loc[:,['accommodates', 'bathrooms', 'bedrooms', 'beds']].describe()

,accommodates,bathrooms,bedrooms,beds
count,20769.000000,20754.000000,20761.000000,20746.000000
mean,3.145794,1.251903,1.324888,1.689723
std,2.034403,0.566285,0.877014,1.119165
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000
50%,2.000000,1.000000,1.000000,1.000000
75%,4.000000,1.000000,2.000000,2.000000
max,16.000000,10.000000,15.000000,17.000000


The data shows that the number of bathrooms, bedrooms and beds are highly right skewed. To explore these outliers:

In [30]:
df.loc[df.accommodates == 16].shape[0]

60

It turns out there are 60 properties that can accommodate up to 16 people. So it is not that unusual.

In [31]:
df.loc[df.bathrooms == 10, ['listing_url','name','accommodates','bathrooms','bedrooms','beds','price']]

,listing_url,name,accommodates,bathrooms,bedrooms,beds,price
20422,https://www.airbnb.com/rooms/35063046,Modern Comfy in Downtown Toronto,12,10.0,12.0,12.0,75.0


In [32]:
df.loc[df.bedrooms == 15, ['listing_url','name','bathrooms','bedrooms','beds','price']]

,listing_url,name,bathrooms,bedrooms,beds,price
16391,https://www.airbnb.com/rooms/30796225,"4 LOFTS- 6000 SQ FT-2-15 bedrooms, 8 bathrooms",8.0,15.0,15.0,1000.0


In [33]:
df.loc[df.beds == 17, ['listing_url','name','bathrooms','bedrooms','beds','price']]

,listing_url,name,bathrooms,bedrooms,beds,price
16058,https://www.airbnb.com/rooms/30366574,Spacious 13 bedroom House for Large Group & Ev...,6.0,13.0,17.0,999.0
17704,https://www.airbnb.com/rooms/32394735,Spacious 13 bedroom House for Large Group & Ev...,6.0,13.0,17.0,999.0


The numbers are verified by inspecting the listing websites.

### 4.3.5 Booking Policy

In [34]:
df.loc[:,['guests_included', 'minimum_nights']].describe()

,guests_included,minimum_nights
count,20769.000000,20769.000000
mean,1.698397,6.293033
std,1.380143,29.357247
min,1.000000,1.000000
25%,1.000000,1.000000
50%,1.000000,2.000000
75%,2.000000,3.000000
max,16.000000,1125.000000


The minimum stay of 1125 seems unreasonable. The listing url is visited to investigate further. 

In [35]:
df.loc[df.minimum_nights==1125,['id','listing_url','name','price','minimum_nights']]

,id,listing_url,name,price,minimum_nights
2815,8001458,https://www.airbnb.com/rooms/8001458,Clean private ensuite with parking,75.0,1125
4994,13883417,https://www.airbnb.com/rooms/13883417,"Bright, modern condo in hip King West downtown",101.0,1125


There are 2 listings with this minimum nights of stay. In the first listing (id# 8001458), the minimum nights of stay is listed as 1125, while no information is provided in the second listing.  

It is likely that the host is relocated for prolonged period, so they require this unusually long stay.  They are believed to be neither outliers nor data inaccuracy.

### 4.3.6 Availability

In [36]:
df.loc[:,['availability_30', 'availability_60', 'availability_90', 'availability_365']].describe()

,availability_30,availability_60,availability_90,availability_365
count,20769.000000,20769.000000,20769.000000,20769.000000
mean,8.250710,19.985074,33.809861,120.708797
std,9.973172,21.021136,32.376896,124.265379
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,4.000000,12.000000,27.000000,79.000000
75%,14.000000,37.000000,64.000000,213.000000
max,30.000000,60.000000,90.000000,365.000000


The values seem legit.

### 4.3.7 Reviews

In [37]:
df.loc[:,['number_of_reviews', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']].describe()

,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
count,20769.000000,20769.000000,16497.000000,16487.000000,16489.000000,16486.000000,16489.000000,16481.000000,16484.00000,16786.000000
mean,25.531224,12.398864,94.437473,9.660278,9.434108,9.736443,9.760143,9.704448,9.48362,1.865878
std,46.686871,20.487604,8.023803,0.788436,0.946426,0.712718,0.705147,0.655603,0.85127,2.142205
min,0.000000,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,0.010000
25%,1.000000,0.000000,93.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.00000,0.350000
50%,7.000000,3.000000,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,1.010000
75%,28.000000,16.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,2.660000
max,717.000000,185.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,17.140000


All values seems legit.

### 4.4 Missing values imputation 

In this section, the missing values of the both numeric and categorical features will be addressed. Again, only features likely relevant to data analysis will be investigated.

### 4.4.1 Numeric features

In [38]:
num_with_na = []
numeric_features = [col for col in df.columns if df[col].dtype in ['int64','float64']]

for col in numeric_features:
    if df[col].isnull().sum() > 0:
        print(f'{col} has {df[col].isnull().sum()} missing values')
        num_with_na.append(col)

host_response_rate has 4984 missing values
host_listings_count has 5 missing values
host_total_listings_count has 5 missing values
bathrooms has 15 missing values
bedrooms has 8 missing values
beds has 23 missing values
security_deposit has 4902 missing values
cleaning_fee has 3384 missing values
review_scores_rating has 4272 missing values
review_scores_accuracy has 4282 missing values
review_scores_cleanliness has 4280 missing values
review_scores_checkin has 4283 missing values
review_scores_communication has 4280 missing values
review_scores_location has 4288 missing values
review_scores_value has 4285 missing values
reviews_per_month has 3983 missing values


It is uncertain why the features host listings count, host response rate, bathrooms, bedrooms, beds are missing.   
To impute those features, the following steps are used:  
    1. Create a new column to record the rows with missing values (with name [COL]_NA)
    2. Impute columns with column median

In [39]:
for col in ['host_listings_count','bathrooms','host_response_rate','bedrooms','beds']:
    df[col+'_NA'] = np.where(df[col].isnull(),True, False)
    m = df[col].median()
    df[col].fillna(m,inplace=True)

Missing security deposits and cleaning fees are likely due the hosts' decision to not include one, which is equivalent to a value of 0. Io impute, the following stems are used:
    1. Create a new column to record the rows with missing values (with name [COL]_NA)
    2. Impute columns with value of 0

In [40]:
for col in ['security_deposit','cleaning_fee']:
    df[col+'_NA'] = np.where(df[col].isnull(),True, False)
    df[col].fillna(0,inplace=True)

For review scores, the missing values are likely due to the fact that the listings are new with no or few customers. To impute, the following steps are used:
    1. Create a new column to record the rows with missing values (with name [COL]_NA)
    2. Impute columns with median

In [41]:
for col in ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
           'review_scores_communication','review_scores_location','review_scores_value','reviews_per_month']:
    df[col+'_NA'] = np.where(df[col].isnull(),True, False)
    df[col].fillna(df[col].median(),inplace=True)

### 4.4.2 Categorical Features

In [42]:
#generate column list for category features
cat_features = [col for col in df.columns if df[col].dtype.name == "category"]
print(cat_features)

['host_response_time', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'is_location_exact', 'property_type', 'room_type', 'bed_type', 'has_availability', 'requires_license', 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification']


In [43]:
cat_with_na = []
for col in cat_features:
    if df[col].isnull().sum() > 0:
        print(f'{col} has {df[col].isnull().sum()} missing values')
        cat_with_na.append(col)

host_response_time has 4984 missing values
host_is_superhost has 5 missing values
host_has_profile_pic has 5 missing values
host_identity_verified has 5 missing values


To impute for host response time, the following steps are used:
    1. Create a new column to record the rows with missing values (with name [COL]_NA)
    2. Impute columns with mode

In [44]:
for col in cat_with_na:
    df[col+"_NA"] = np.where(df[col].isnull(),1,0)
    df[col].fillna(df[col].mode().iloc[0],inplace=True)

### 4.4.3 Datetime Features 

In [45]:
df.host_since.dtypes

dtype('<M8[ns]')

In [46]:
#generate column list for datetime features
dt_features = [col for col in df.columns if df[col].dtype == "<M8[ns]"]
print(dt_features)

['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review', 'last_review']


In [47]:
dt_with_na = []
for col in dt_features:
    if df[col].isnull().sum() > 0:
        print(f'{col} has {df[col].isnull().sum()} missing values')
        dt_with_na.append(col)

host_since has 5 missing values
first_review has 3983 missing values
last_review has 3983 missing values


To impute for the missing values, the following steps are used:
    1. Create a new column to record the rows with missing values (with name [COL]_NA)
    2. Impute columns with median (quantile = 0.5)

In [48]:
for col in dt_with_na:
    df[col+"_NA"] = np.where(df[col].isnull(),1,0)
    df[col].fillna(df[col].quantile(0.5),inplace=True)

# 5 New Feature Creation

## 5.1 Dummy Variable for Amenities

The column _amenities_ contains a string of available amenities for reach listing.

In [49]:
df.amenities.head()

0    {TV,Internet,Wifi,"Air conditioning",Kitchen,"...
1    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
2    {Internet,Wifi,"Pets live on this property",Ca...
3    {Internet,Wifi,"Air conditioning",Kitchen,"Fre...
4    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
Name: amenities, dtype: object

First, create a new column with the string converted to a list:

In [50]:
df['amenities_list'] = df['amenities'].str.lower().str.replace('[{}"]','').str.split(',')

In [51]:
df['amenities_list'].head()

0    [tv, internet, wifi, air conditioning, kitchen...
1    [tv, cable tv, internet, wifi, air conditionin...
2    [internet, wifi, pets live on this property, c...
3    [internet, wifi, air conditioning, kitchen, fr...
4    [tv, cable tv, internet, wifi, air conditionin...
Name: amenities_list, dtype: object

In [52]:
#Create amenities set
amenities_set = set()
for i in df['amenities_list']:
    amenities_set.update(i)
print(amenities_set)

{'wide clearance to shower', '', 'other pet(s)', 'internet', 'brick oven', 'baby bath', 'outdoor seating', 'beachfront', '24-hour check-in', 'bidet', 'fire extinguisher', 'hammock', 'carbon monoxide detector', 'private bathroom', 'ceiling fan', 'heated towel rack', 'pool', 'hangers', 'paid parking off premises', 'breakfast table', 'espresso machine', 'en suite bathroom', 'dryer', 'laptop friendly workspace', 'netflix', 'fixed grab bars for toilet', 'game console', 'fire pit', 'mini fridge', 'lock on bedroom door', 'sun loungers', 'other', 'table corner guards', 'pocket wifi', 'central air conditioning', 'private living room', 'breakfast', 'disabled parking spot', 'doorman', 'bathtub with bath chair', 'stair gates', 'ski-in/ski-out', 'hot tub', 'murphy bed', 'self check-in', 'double oven', 'shampoo', 'well-lit path to entrance', 'memory foam mattress', 'wide hallways', 'printer', 'iron', 'alfresco bathtub', 'high-resolution computer monitor', 'first aid kit', 'shower chair', 'cleaning b

In [53]:
len(amenities_set)

197

In total, there are 197 unique amenities. There one with null entry (1st item) which will be removed.

In [54]:
amenities_set.remove('')

Next, dummy variables are created for each item and each listing. Values of _True_ and _False_ will be assigned according to their availability.

In [55]:
for item in amenities_set:
    df['amenities_' + item] = df.amenities_list.apply(lambda x: 1 if item in x else 0) 

In [56]:
for item in amenities_set:
    print(f'Number of listings with {item}: {df["amenities_" + item].sum()}')

Number of listings with wide clearance to shower: 387
Number of listings with other pet(s): 57
Number of listings with internet: 5002
Number of listings with brick oven: 1
Number of listings with baby bath: 182
Number of listings with outdoor seating: 76
Number of listings with beachfront: 137
Number of listings with 24-hour check-in: 1959
Number of listings with bidet: 6
Number of listings with fire extinguisher: 10749
Number of listings with hammock: 1
Number of listings with carbon monoxide detector: 16672
Number of listings with private bathroom: 7
Number of listings with ceiling fan: 39
Number of listings with heated towel rack: 7
Number of listings with pool: 3822
Number of listings with hangers: 17605
Number of listings with paid parking off premises: 4761
Number of listings with breakfast table: 91
Number of listings with espresso machine: 56
Number of listings with en suite bathroom: 78
Number of listings with dryer: 16557
Number of listings with laptop friendly workspace: 156

In total, 196 new columns are created.

## 5.2 City Names

In addition to _longitude_ and _latitude_, some other columns may also contain useful geographical information, namely _street_, _city_, _zipcode_, _market_, and *smart_location*.

In [57]:
df.street.unique()

array(['Toronto, ON, Canada', 'toronto, ON, Canada', 'Toronto, 1, Canada',
       'Guelph, ON, Canada', 'Etobicoke, ON, Canada',
       'Toronto, Ont, Canada', 'Toronto, ON, ON, Canada',
       'North York, Toronto, ON, Canada', '토론토, 온타리오, Canada',
       'Scarborough, ON, Canada', '多伦多, 安大略省, Canada',
       'North York, ON, Canada', 'Vaughan, ON, Canada', 'Toronto, Canada',
       'scarborough, ON, Canada', 'Toronto, On, Canada',
       'Etobicoke , ON, Canada', 'Mississauga, ON, Canada',
       'Toronto, , ON, Canada', 'North york, ON, Canada',
       'Toronto , ON, Canada', '多伦多, ON, Canada',
       'Toronto, Ontario, CA, ON, Canada', 'North York , ON, Canada',
       'SCARBOROUGH, ON, Canada', 'York, ON, Canada',
       'Markham, ON, Canada', 'Toronto M5S 2L6, ON, Canada',
       'Scarborough , ON, Canada', 'ONTARIO , TORONTO, Canada',
       'Vaughan , ON, Canada', 'North york - Toronto , ON, Canada',
       'Scarborough , Toronto  , ON, Canada', 'toronto, ontario, Canada',
    

In [58]:
df.city.unique()

array(['Toronto', 'toronto', 'Guelph', 'Etobicoke', 'Toronto, ON',
       'North York, Toronto', '토론토', 'Scarborough', '多伦多', 'North York',
       'Vaughan', 'scarborough', 'Etobicoke ', 'Mississauga', 'Toronto, ',
       'North york', 'Toronto ', 'Toronto, Ontario, CA', 'North York ',
       'SCARBOROUGH', 'York', 'Markham', 'Toronto M5S 2L6', 'Scarborough ',
       'ONTARIO ', 'Vaughan ', 'North york - Toronto ',
       'Scarborough , Toronto  ', nan, 'Thornhill ', 'Pickering'], dtype=object)

In [59]:
df.market.unique()

array(['Toronto', 'Montreal', 'Paris', nan, 'Other (International)'], dtype=object)

In [60]:
df.smart_location.unique()

array(['Toronto, Canada', 'toronto, Canada', 'Guelph, Canada',
       'Etobicoke, Canada', 'Toronto, ON, Canada',
       'North York, Toronto, Canada', '토론토, Canada', 'Scarborough, Canada',
       '多伦多, Canada', 'North York, Canada', 'Vaughan, Canada',
       'scarborough, Canada', 'Etobicoke , Canada', 'Mississauga, Canada',
       'Toronto, , Canada', 'North york, Canada', 'Toronto , Canada',
       'Toronto, Ontario, CA, Canada', 'North York , Canada',
       'SCARBOROUGH, Canada', 'York, Canada', 'Markham, Canada',
       'Toronto M5S 2L6, Canada', 'Scarborough , Canada',
       'ONTARIO , Canada', 'Vaughan , Canada',
       'North york - Toronto , Canada', 'Scarborough , Toronto  , Canada',
       'ON, Canada', 'Thornhill , Canada', 'Pickering, Canada'], dtype=object)

The features of *street*, *city*, *market* and *smart_location* are not all that useful.

Explore *zipcode*:

In [61]:
df.zipcode.nunique()

6925

In [62]:
# Top 20 zipcode by count
df.zipcode.value_counts(dropna=False).head(20)

NaN        365
M5J 0B1    344
M5R 1C4    199
M5V        191
M5V 0E9    159
M5J        158
M5V 4A9    130
M5J 0A6    108
M5V 0J9     99
M5V 0G6     96
M5J 0A7     96
M5V 4A2     93
M5V 4B2     91
M6J         90
M5V 0J8     77
M5J 2Z2     76
M5V 4A5     73
M5H 0B1     72
M2N         65
M5J 0A1     62
Name: zipcode, dtype: int64

The zipcode consists of 365 missing values. Also, it consists of both the 6-digit format (FSA + LDU) and the 3-digit format. FSA stands for **Forward Sortation Area** and LDU stands for **Local Delivery Unit**. The FSA can be used to identify the city of the listing. 

First, a new column with just the fsa is created:  

In [7]:
df['fsa'] = df.zipcode.str.slice(stop=3).str.upper()

In [8]:
df['fsa'].nunique()

116

In [65]:
# display top 30 FSA by count
df['fsa'].value_counts(dropna=False).head(30)

M5V    2696
M5J    1189
M6J     998
M6K     810
M2N     761
M6H     668
M5R     660
M5A     641
M4Y     635
M6G     595
M5T     566
M6P     394
M5S     387
M5B     368
NaN     365
M4M     331
M6R     318
M4L     311
M2M     287
M4C     272
M2J     265
M4J     258
M4K     248
M6E     226
M4X     226
M8V     223
M3J     220
M5H     182
M4E     179
M4S     179
Name: fsa, dtype: int64

Next, the FSA values are matched with the City. The website https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M contains the FSA codes along with the corresponding regions in Greater Toronto Area.  The data table was copied and pasted into a csv file. 

The file *toronto_fsa.csv* can be downloaded from https://drive.google.com/open?id=1-ImaFEoNC-TQji5LxDtAdH7jlJ2q1A0P

In [5]:
toronto_fsa = pd.read_csv('data/toronto_fsa.csv')
toronto_fsa.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
#create a dictionary with postcode as key and borough (city) as dictionary
fsa_dict = {i:j for i,j in zip(toronto_fsa.Postcode, toronto_fsa.Borough)}

#create new column that maps the FSA to the city name
df['city_fsa'] = np.nan
df['city_fsa'] = df.fsa.map(fsa_dict)

In [10]:
# The counts for each assigned city
df.city_fsa.value_counts(dropna=False)

Downtown Toronto    7938
West Toronto        3361
North York          2749
Central Toronto     1426
Scarborough         1359
Etobicoke           1083
East Toronto        1069
York                 707
East York            671
NaN                  376
Thornhill             20
Pickering              2
Not assigned           2
Mississauga            2
Markham                1
Kleinburg              1
Guelph                 1
Queen's Park           1
Name: city_fsa, dtype: int64

In [14]:
df.loc[df.city_fsa == 'Guelph',['longitude','latitude']]

,longitude,latitude
162,-79.48815,43.62575


In [69]:
# The number of listings not assigned a city
df.city_fsa.isnull().sum()

403

In [70]:
df.loc[df.city_fsa.isnull(),'fsa'].value_counts(dropna=False)

NaN    365
L3T     12
L4J      8
[NO      3
L1W      2
M        2
2V5      1
SEC      1
M15      1
ON       1
L0J      1
N1E      1
L5E      1
L3S      1
L4W      1
G7H      1
MP6      1
Name: fsa, dtype: int64

With unknown or ambiguous zipcode, it is difficult to locate the property. To impute the missing *city_fsa* values:
    1. A column to identify the missing value is created
    2. The missing value is imputed with 'Other'

In [71]:
df['city_fsa_NA'] = np.where(df['city_fsa'].isnull(),True,False)
df['city_fsa'].fillna('Other',inplace=True)

## 5.3 Date time Difference from Reference Day

The number of months since the recorded events can be a feature more useful than the dates. A reference date of 2019/6/27, which is the date this dataset was scrapped is chosen.

In [72]:
ref_date = datetime.datetime(2019,6,27)

In [73]:
(ref_date - df['host_since'][1]).days

3657

In [74]:
dt_col = ['host_since','first_review','last_review']
for col in dt_col:
    df['days_since'+col] = df[col].apply(lambda x: (ref_date - x).days)

In [ ]:
df.shape

In [ ]:
df.head()

In [80]:
df[df.availability_365==0].shape[0]

5212

In [84]:
df.loc[5278,['listing_url','price','cleaning_fee']]

listing_url     https://www.airbnb.com/rooms/14230700
price                                              40
cleaning_fee                                      800
Name: 5278, dtype: object

In [77]:
df.loc[df.cleaning_fee_NA==True, ['listing_url','price','minimum_nights','availability_365']]

,listing_url,price,minimum_nights,availability_365
1,https://www.airbnb.com/rooms/8077,100.0,180,0
5,https://www.airbnb.com/rooms/27423,54.0,120,0
6,https://www.airbnb.com/rooms/30931,134.0,180,365
18,https://www.airbnb.com/rooms/62545,64.0,60,279
21,https://www.airbnb.com/rooms/64641,64.0,50,305
43,https://www.airbnb.com/rooms/148945,81.0,3,251
51,https://www.airbnb.com/rooms/183522,179.0,120,82
55,https://www.airbnb.com/rooms/196360,48.0,29,297
60,https://www.airbnb.com/rooms/218804,87.0,2,269
72,https://www.airbnb.com/rooms/261235,66.0,1,289


# 6 Removal Irrelevant Variables

As the final step, features that are likely irrelevant for data analysis are removed.

In [ ]:
#remove irrelevant string columns
l_str = []
for col in df.columns:
    if df[col].dtype == 'O':
        l_str.append(col)
print(l_str)

In [ ]:
col_remove_str = ['listing_url', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 
              'transit', 'access', 'interaction', 'house_rules', 'picture_url', 'host_url', 'host_name', 'host_location', 
              'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'street', 
              'neighbourhood', 'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code',
              'country', 'amenities', 'calendar_updated', 'amenities_list', 'fsa']
df.drop(col_remove_str,axis=1,inplace=True)
removed_columns_record.append(col_remove_str)

In [ ]:
#remove irrelevant numeric columns

l_num = []
for col in df.columns:
    if df[col].dtype.name in ['int64','float64']:
        l_num.append(col)
print(l_num)

In [ ]:
# col_remove_num = ['id', 'scrape_id', 'host_id','minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
#                   'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']
# df.drop(col_remove_num,axis=1,inplace=True)
# removed_columns_record.append(col_remove_num)

In [ ]:
#remove irrelevant datetime variables

# l_dt = []
# for col in df.columns:
#     if df[col].dtype == '<M8[ns]':
#         l_dt.append(col)
# print(l_dt)

In [ ]:
# col_remove_dt = ['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review', 'last_review']
# df.drop(col_remove_dt,axis=1,inplace=True)
# removed_columns_record.append(col_remove_dt)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_excel('data/df_cleaned.xlsx',index=False)

In [ ]:
df.dtypes

In [ ]:
# Export data
df.to_csv('data/df_cleaned.csv',index=False,encoding='utf-8')

In [ ]:
df_csv = pd.read_csv('data/df_cleaned.csv',encoding='utf-8')

In [ ]:
df_csv.head()

In [ ]:
df_ = pd.read_excel('data/df_cleaned.xlsx')

In [ ]:
df_cleaned.head()

In [ ]:
df.loc[df.cleaning_fee_NA==True,['listing_url','price','cleaning_fee_NA']]

# 7. Conclusion

In this section, the following data wrangling steps where performed:
    1. Remove columns with excessive missing values;
    2. Inspect and address outliers;
    3. Impute data;
    4. Create new features; and
    5. Remove irrelevant features
The next step of this project is data visualization which will be presented in subsequent notebooks.